# Chicago Education and Crime Data Management System

In [32]:
import pandas as pd
import sqlite3

census_data = pd.read_csv('ChicagoCensusData.csv')
public_schools = pd.read_csv('ChicagoPublicSchools.csv')
crime_data = pd.read_csv('ChicagoCrimeData.csv')

conn = sqlite3.connect('FinalDB.db')

census_data.to_sql('CENSUS_DATA', conn, if_exists='replace', index=False)
public_schools.to_sql('CHICAGO_PUBLIC_SCHOOLS', conn, if_exists='replace', index=False)
crime_data.to_sql('CHICAGO_CRIME_DATA', conn, if_exists='replace', index=False)

%load_ext sql
%sql sqlite:///FinalDB.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


### Exercise 1, Question 1: 

Write and execute a SQL query to list the school names, community names and average attendance for communities with a hardship index of 98. 

In [13]:
%%sql
SELECT cps.NAME_OF_SCHOOL, census.COMMUNITY_AREA_NAME, AVG(cps.AVERAGE_STUDENT_ATTENDANCE) AS AVG_ATTENDANCE 
FROM CENSUS_DATA census
JOIN CHICAGO_PUBLIC_SCHOOLS cps ON census.COMMUNITY_AREA_NUMBER = cps.COMMUNITY_AREA_NUMBER
WHERE census.HARDSHIP_INDEX = 98
GROUP BY cps.NAME_OF_SCHOOL, census.COMMUNITY_AREA_NAME;

 * sqlite:///FinalDB.db
Done.


NAME_OF_SCHOOL,COMMUNITY_AREA_NAME,AVG_ATTENDANCE
George Washington Carver Military Academy High School,Riverdale,91.6
George Washington Carver Primary School,Riverdale,90.9
Ira F Aldridge Elementary School,Riverdale,92.9
William E B Dubois Elementary School,Riverdale,93.3


### Exercise 1, Question 2: 

Write and execute a SQL query to list all crimes that took place at a school. Include case number, crime type and community name.

In [25]:
%%sql
SELECT c.CASE_NUMBER, c.PRIMARY_TYPE, cd.COMMUNITY_AREA_NAME
FROM CHICAGO_CRIME_DATA c
JOIN CHICAGO_PUBLIC_SCHOOLS cps ON c.BLOCK LIKE '%' || cps.STREET_ADDRESS || '%'
JOIN CENSUS_DATA cd ON cps.COMMUNITY_AREA_NUMBER = cd.COMMUNITY_AREA_NUMBER
WHERE c.BLOCK LIKE '%' || 'SCHOOL' || '%' OR c.BLOCK LIKE '%' || 'HS' || '%'

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER,PRIMARY_TYPE,COMMUNITY_AREA_NAME


### Exercise 2, Question 1:
Write and execute a SQL statement that returns just the school name and leaders’ icon from the view.

In [37]:
%%sql
SELECT SCHOOL_NAME, LEADERS_ICON
FROM CPS_SCHOOLS_WITH_LEADERS;

 * sqlite:///FinalDB.db
(sqlite3.OperationalError) no such table: CPS_SCHOOLS_WITH_LEADERS
[SQL: SELECT SCHOOL_NAME, LEADERS_ICON
FROM CPS_SCHOOLS_WITH_LEADERS;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


### Exercise 3, Question 1: 

Write the structure of a query to create or replace a stored procedure called UPDATE_LEADERS_SCORE that takes a in_School_ID parameter as an integer and a in_Leader_Score parameter as an integer. Don't forget to use the #SET TERMINATOR statement to use the @ for the CREATE statement terminator. 

In [38]:
%%sql
#SET TERMINATOR @

CREATE OR REPLACE PROCEDURE UPDATE_LEADERS_SCORE (IN in_School_ID INT, IN in_Leader_Score INT)
BEGIN
    -- code to update the Leaders_Score field
END
@

 * sqlite:///FinalDB.db
(sqlite3.OperationalError) near "#SET": syntax error
[SQL: #SET TERMINATOR @

CREATE OR REPLACE PROCEDURE UPDATE_LEADERS_SCORE (
    IN in_School_ID INTEGER,
    IN in_Leader_Score INTEGER
)
BEGIN
    -- SQL statement to update Leaders_Score field
END
@]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


### Exercise 3, Question 2: 

Inside your stored procedure, write a SQL statement to update the Leaders_Score field in the CHICAGO_PUBLIC_SCHOOLS table for the school identified by in_School_ID to the value in the in_Leader_Score parameter. 

In [41]:
%%sql
--#SET TERMINATOR @

CREATE OR REPLACE PROCEDURE UPDATE_LEADERS_SCORE (IN in_School_ID INTEGER, IN in_Leader_Score INTEGER)
  LANGUAGE SQL
  
	BEGIN 
	
		UPDATE CHICAGO_PUBLIC_SCHOOLS
		SET "Leaders_Score" = in_Leader_Score
			WHERE "School_ID" = in_School_ID;
			
	END@

 * sqlite:///FinalDB.db
(sqlite3.OperationalError) near "#SET": syntax error
[SQL: #SET TERMINATOR ;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


### Exercise 3, Question 3: 

Inside your stored procedure, write a SQL IF statement to update the Leaders_Icon field in the CHICAGO_PUBLIC_SCHOOLS table for the school identified by in_School_ID using the following information. 

In [42]:
%%sql
CREATE OR REPLACE PROCEDURE UPDATE_LEADERS_SCORE (IN in_School_ID INTEGER, IN in_Leader_Score INTEGER)
  LANGUAGE SQL
  
	BEGIN 

		UPDATE CHICAGO_PUBLIC_SCHOOLS
		SET "Leaders_Score" = in_Leader_Score
			WHERE "School_ID" = in_School_ID;
			
		IF in_Leader_Score > 0 AND in_Leader_Score < 20 THEN
	      	UPDATE CHICAGO_PUBLIC_SCHOOLS
				SET "Leaders_Icon" = 'Very Weak';
	    ELSEIF in_Leader_Score < 40 THEN
	       	UPDATE CHICAGO_PUBLIC_SCHOOLS
				SET "Leaders_Icon" = 'Weak';	
	    ELSEIF in_Leader_Score < 60 THEN
	       	UPDATE CHICAGO_PUBLIC_SCHOOLS
				SET "Leaders_Icon" = 'Average';
	    ELSEIF in_Leader_Score < 80 THEN
	       	UPDATE CHICAGO_PUBLIC_SCHOOLS
				SET "Leaders_Icon" = 'Strong';
	    ELSEIF in_Leader_Score < 100 THEN
	       	UPDATE CHICAGO_PUBLIC_SCHOOLS
				SET "Leaders_Icon" = 'Very Strong';
	   	END IF;
		
	END@

 * sqlite:///FinalDB.db
(sqlite3.OperationalError) near "IF": syntax error
[SQL: IF in_Leader_Score >= 0 AND in_Leader_Score < 20 THEN
    UPDATE CHICAGO_PUBLIC_SCHOOLS SET Leaders_Icon = 'very weak' WHERE SCHOOL_ID = in_School_ID;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


### Exercise 3, Question 4: 

Run your code to create the stored procedure.

In [43]:
%%sql
CREATE OR REPLACE PROCEDURE UPDATE_LEADERS_SCORE (IN in_School_ID INTEGER, IN in_Leader_Score INTEGER)
  LANGUAGE SQL
  
	BEGIN 

		UPDATE CHICAGO_PUBLIC_SCHOOLS
		SET "Leaders_Score" = in_Leader_Score
			WHERE "School_ID" = in_School_ID;
			
		IF in_Leader_Score > 0 AND in_Leader_Score < 20 THEN
	      	UPDATE CHICAGO_PUBLIC_SCHOOLS
				SET "Leaders_Icon" = 'Very Weak';
	    ELSEIF in_Leader_Score < 40 THEN
	       	UPDATE CHICAGO_PUBLIC_SCHOOLS
				SET "Leaders_Icon" = 'Weak';	
	    ELSEIF in_Leader_Score < 60 THEN
	       	UPDATE CHICAGO_PUBLIC_SCHOOLS
				SET "Leaders_Icon" = 'Average';
	    ELSEIF in_Leader_Score < 80 THEN
	       	UPDATE CHICAGO_PUBLIC_SCHOOLS
				SET "Leaders_Icon" = 'Strong';
	    ELSEIF in_Leader_Score < 100 THEN
	       	UPDATE CHICAGO_PUBLIC_SCHOOLS
				SET "Leaders_Icon" = 'Very Strong';
	   	END IF;
		
	END@

 * sqlite:///FinalDB.db
(sqlite3.OperationalError) near "CALL": syntax error
[SQL: CALL UPDATE_LEADERS_SCORE(1234, 75);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


### Exercise 4, Question 1: 

Update your stored procedure definition. Add a generic ELSE clause to the IF statement that rolls back the current work if the score did not fit any of the preceding categories. 

In [44]:
%%sql
CREATE PROCEDURE score_classifier(score float)
BEGIN
	IF score >= 0 AND score < 50 THEN
		UPDATE scores SET class = 'Low' WHERE score = score_id;
	ELSEIF score >= 50 AND score < 80 THEN
		UPDATE scores SET class = 'Medium' WHERE score = score_id;
	ELSEIF score >= 80 AND score <= 100 THEN
		UPDATE scores SET class = 'High' WHERE score = score_id;
	ELSE
		ROLLBACK;
	END IF;
END;

 * sqlite:///FinalDB.db
(sqlite3.OperationalError) near "PROCEDURE": syntax error
[SQL: CREATE PROCEDURE score_classifier(score float)
BEGIN
	IF score >= 0 AND score < 50 THEN
		UPDATE scores SET class = 'Low' WHERE score = score_id;
	ELSEIF score >= 50 AND score < 80 THEN
		UPDATE scores SET class = 'Medium' WHERE score = score_id;
	ELSEIF score >= 80 AND score <= 100 THEN
		UPDATE scores SET class = 'High' WHERE score = score_id;
	ELSE
		ROLLBACK;
	END IF;
END;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


### Exercise 4, Question 2: 

Update your stored procedure definition again. Add a statement to commit the current unit of work at the end of the procedure. 

In [45]:
%%sql
CREATE PROCEDURE score_classifier(score float)
BEGIN
	IF score >= 0 AND score < 50 THEN
		UPDATE scores SET class = 'Low' WHERE score = score_id;
	ELSEIF score >= 50 AND score < 80 THEN
		UPDATE scores SET class = 'Medium' WHERE score = score_id;
	ELSEIF score >= 80 AND score <= 100 THEN
		UPDATE scores SET class = 'High' WHERE score = score_id;
	ELSE
		ROLLBACK;
	END IF;
	
	COMMIT;
END;


 * sqlite:///FinalDB.db
(sqlite3.OperationalError) near "PROCEDURE": syntax error
[SQL: CREATE PROCEDURE score_classifier(score float)
BEGIN
	IF score >= 0 AND score < 50 THEN
		UPDATE scores SET class = 'Low' WHERE score = score_id;
	ELSEIF score >= 50 AND score < 80 THEN
		UPDATE scores SET class = 'Medium' WHERE score = score_id;
	ELSEIF score >= 80 AND score <= 100 THEN
		UPDATE scores SET class = 'High' WHERE score = score_id;
	ELSE
		ROLLBACK;
	END IF;
	
	COMMIT;
END;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
